# Imports

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# %matplotlib qt
import generate_data as gd
import EDXS_model as em
import numpy as np
import hyperspy.api as hs
import matplotlib.pyplot as plt

# Parameters

In [ ]:
# To save the dataset
filename = "aspim037_N100_2ptcls_brstlg"
db_path = "Data/simple_xrays_threshold.json"
abs_db_path = None
#abs_db_path = "Data/wernisch_abs.json"
abs_elt_dict = None

# Continuum X-rays parameters
# They were determine by fitting experimental data from 0.6 to 18 keV. Since low energies were incorporated, the model is only effective and not quantitative. 
brstlg_pars = {"c0" : 0.094,"c1" : 1417, "c2" : 1e-6, "b0" : 1.2, "b1" : -0.06, "b2" : 0.00683}

#brstlg_pars = {"c0" : 9.3144e-04,"c1" : 1.23453632, "c2" : 4.9488e-10, "b0" : 0.14986191, "b1" : -0.00188483, "b2" : 2.5428e-04}


scale = 1

# Average number of counts in one spectrum of the artificial data
N = 100 

# Creation of the pure spectra of the different phases.
phase1 = em.EDXS_Model(db_path,abs_db_path,brstlg_pars)
#phase1.generate_abs_coeff({"8":1.0,"12" : 0.51,"14":0.61,"13":0.07,"20":0.04,"62":0.02,"26":0.028,"60":0.002,"71":0.003,"72":0.003,"29" : 0.02})
# Gaussians corresponding to elements
phase1.generate_spectrum({"8":1.0,"12" : 0.51,"14":0.61,"13":0.07,"20":0.04,"62":0.02,"26":0.028,"60":0.002,"71":0.003,"72":0.003,"29" : 0.02},scale)

phase2 = em.EDXS_Model(db_path,abs_db_path,brstlg_pars)
#phase2.generate_abs_coeff({"8":0.54,"26":0.15,"12" : 1.0,"29":0.038,"92":0.0052,"60":0.004,"31":0.03,"71":0.003})
phase2.generate_spectrum({"8":0.54,"26":0.15,"12" : 1.0,"29":0.038,"92":0.0052,"60":0.004,"31":0.03,"71":0.003},scale)

phase3 = em.EDXS_Model(db_path,abs_db_path,brstlg_pars)
#phase3.generate_abs_coeff({"8":1.0,"14":0.12,"13":0.18,"20":0.47,"62":0.04,"26":0.004,"60":0.008,"72":0.004,"29":0.01})
phase3.generate_spectrum({"8":1.0,"14":0.12,"13":0.18,"20":0.47,"62":0.04,"26":0.004,"60":0.008,"72":0.004,"29":0.01},scale)

# Objects needed for the creation of data
# list of spectra
phases = np.array([phase1.spectrum,phase2.spectrum,phase3.spectrum])
# list of densities which will give different total number of events per spectra
densities = np.array([1.0,1.33,1.25])

In [ ]:
phases.shape

# Artificial data creation

Note : Each function of ArtificialSpim updates the current object and adds on top. It is usually better to re-create a new object to change the artificial data settings.
Spim corresponds to spectrum image.

In [ ]:
# Creation of an ArtificialSpim instance
spim=gd.AritificialSpim(phases,densities,(80,80))

In [ ]:
# We add two particles belonging to two different phases to the data
spim.sphere((25,30),3.5,3.5,0.0,0.5,1)
spim.sphere((55,30),3.5,3.5,0.0,0.5,2)

In [ ]:
plt.figure(figsize=(15, 4))
for i in range(3):
    plt.subplot(1,3,i+1)
    plt.imshow(spim.weights[:,:,i], vmin=0, vmax=1)
    plt.colorbar()

In [ ]:
# Generates a noiseless version of the dataset
Y = spim.generate_spim_deterministic()
X = spim.generated_spim
Y.shape, X.shape # What is Y?

In [ ]:
# Generates a noiseless version of the dataset
Y2 = spim.generate_spim_deterministic()
X2 = spim.generated_spim
Y2.shape, X2.shape # What is Y?

In [ ]:
np.random.poisson(10)

In [ ]:
broadcast_matrix.shape

In [ ]:
broadcast_matrix.shape

In [ ]:
plt.imshow(sum)
plt.colorbar()

In [ ]:
densities.shape

In [ ]:
# Generate noisy spectra based on a Poisson distribution with average N
spim.generate_spim_stochastic(N)

In [ ]:
# To save the data and the ground truth under similar names
spim.save(filename)

# Data visualisation

In [ ]:
# To visualise the pure spectrum of the first phase
x = phase1.x
plt.plot(x,phases[0],"g-",linewidth=2)
plt.plot(x,phase1.continuum_xrays())
plt.xlabel("Energy [keV]")
plt.ylabel("Intensity")
plt.rcParams.update({'font.size': 22})
plt.tight_layout()

## Interactive visualization of the created datacube

3 windows will pop : 

1) Map of the sum of counts per pixel - A green square will be plotted as well at the top left

2) Spectrum of the top left corner pixel (hyperspy built-in)

3) Sum of spectra of the green area in 1)

It possible to move and reshape the green area in 1) to explore the data

In [ ]:
signal=hs.signals.Signal1D(spim.stochastic_spim)
roi=hs.roi.RectangularROI(12,12,24,24)
signal.plot()
spim_ROI=roi.interactive(signal)
    
sum_ROI=hs.interactive(spim_ROI.sum,
               event=spim_ROI.axes_manager.events.any_axis_changed,
               recompute_out_event=None)
sum_ROI.plot()